# Run PEST with BeoPEST

### This should be super easy...

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import os
import sys

import pyemu
from apexmf_pkgs.apexmf_pst_utils import execute_beopest, execute_workers
from apexmf_pkgs.apexmf_pst_viz import phi_progress_plot
# from apexmf_pst_pkgs.sm_pst_stats import create_param_unc



# 1. Specify a path to your main

In [ ]:
wd = "D:/Workshops/2022_Webinar_apexmf_opt_BLM/apexmf_1st_cal/APEX-MODFLOW"
os.chdir(wd)

# 2. Initial run for adjusting weights

In [ ]:
os.system("start cmd /k i64pest {0}".format("ani_pest.pst"))

# 3. Reweight

In [ ]:
casename = "ani_pest"
pestoutfile = "ani_pest_rw.pst"
contrib = 4240.7

In [ ]:
os.system("start cmd /k i64pwtadj1 {0} {1} {2}".format(casename, pestoutfile, contrib))

Go check the pst file

# 4. Run Parallel PEST with BeoPEST

In [ ]:
execute_beopest?

In [ ]:
execute_beopest(wd, pestoutfile, reuse_workers=True, num_workers=5, restart=None)

# 5. Run or add workers separately

In [ ]:
execute_workers?

In [ ]:
execute_workers(
    wd, pestoutfile, "172.20.1.9", num_workers=1, start_id=12,
    # worker_root='..', port=4005,
    reuse_workers=True,
    # copy_files=['model.in', 'model.in.tpl']
    )





In [ ]:
pst = pyemu.Pst(os.path.join(wd,pestoutfile))

In [ ]:
pst.phi

In [ ]:
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
phi_progress_plot(pestoutfile)

In [ ]:
figs = pst.plot(kind="1to1")

#### Statistics - Create the "param.unc" file

- An easy way to formulate a list of prior parameter standard deviations (this being one option for filling a parameter uncertainty file) is to divide the difference between these bounds (or their logs for parameters that are log-transformed) by 4.0. 
- This procedure relies on the the implicit assumption that parameters are (log)normally distributed and that their bounds span their 95% confidence intervals.

In [ ]:
create_param_unc?

In [ ]:
create_param_unc("qsm_reweight.pst")

In [ ]:
test = create_param_unc("qsm_reweight.pst")

#### Calibration with SVD

In [ ]:
execute_beopest(wd, "qsm_svd.pst", 10)

#### Calibration with SVD-Reg

In [ ]:
execute_beopest(wd, "svd-reg.pst", 15)

#### Calibration

In [ ]:
execute_beopest(wd, "qsm_reweight.pst", 10)

In [ ]:
import socket
socket.gethostname()